In [126]:
import json
import logging
import os

import pandas as pd


### Globals

In [127]:
LANG_CODES_TO_NAMES = {
    "en": "English",
    "ru": "Russian",
    "ko": "Korean",
    "cmn-Hant": "Traditional Chinese",
    "ja": "Japanese",
    "de": "German",
    "es": "Spanish",
}
logger = logging.getLogger(__name__)

### Helper functions

In [128]:
# Function to get the script directory
def get_script_dir():
    """Returns the directory where the script is located."""
    dir = os.getcwd()
    if dir.endswith("overrideData"):
        return dir[:-len("overrideData")]
    return dir

# Function to generate file path
def get_file_path(base_dir: str, file: str, lang: str, is_en=False):
    return f"{base_dir}{lang if not is_en else 'en'}/{file}.json"

# Function to load a JSON file
def load_file(base_dir: str, file: str, lang: str, is_en=False):
    path = get_file_path(base_dir, file, lang, is_en)
    logger.info(f"LOADING FILE: ++ {path}")
    with open(path, encoding="utf-8") as f:
        return json.load(f)

# Function to load a DataFrame
def load_df(base_dir: str, file: str, lang: str, is_en=False):
    path = get_file_path(base_dir, file, lang, is_en)
    logger.info(f"LOADING FILE: ++ {path}")
    return pd.read_json(path, encoding="utf-8")


### Setting more globals

In [129]:
cwd = get_script_dir()
base_dir = cwd + "/tables/"
out_file = cwd + "/overrideData/annoints.json"
lang = "en"

Load all tables needed for annoints

In [130]:
blight_results = load_df(base_dir, "BlightCraftingResults", lang)
blight_items = load_df(base_dir, "BlightCraftingItems", lang)
blight_crafting_recipes = load_df(base_dir, "BlightCraftingRecipes", lang).drop(columns=["_index", "Id"])
passive_skills = load_df(base_dir, "PassiveSkills", lang).drop(columns=["_index"])
len(blight_results), len(blight_items), len(blight_crafting_recipes), len(passive_skills)

(1000, 10, 1000, 6189)

Join results and skills on r.passiveskill = p.passiveskillgraphid

In [131]:
joined_results_skills = pd.merge(
    blight_results,
    passive_skills,
    left_on='PassiveSkill',
    right_index=True,
    suffixes=('_results', '_skills')
)[["PassiveSkillGraphId", "Name"]]
joined_results_skills

,PassiveSkillGraphId,Name
0,9736,Insulated Treads
2,42354,Blinding Flash
4,46565,Stance Breaker
6,59636,Open Mind
8,30456,High Alert
...,...,...
994,7809,Wild Storm
995,62439,Enraged Reaver
996,44566,Lightning Rod
998,62034,Prism Guard


In [132]:
blight_crafting_recipes.head()

,BlightCraftingResult,BlightCraftingItems
0,0,"[0, 0, 0]"
1,1,"[0, 0, 1]"
2,2,"[0, 1, 0]"
3,3,"[1, 0, 0]"
4,4,"[0, 1, 1]"


In [133]:
joined_results_crafting = pd.merge(
    blight_crafting_recipes,
    joined_results_skills,
    left_on='BlightCraftingResult',
    right_index=True,
    suffixes=('_results', '_crafting')
)[["PassiveSkillGraphId","BlightCraftingItems","Name"]]
joined_results_crafting

,PassiveSkillGraphId,BlightCraftingItems,Name
0,9736,"[0, 0, 0]",Insulated Treads
2,42354,"[0, 1, 0]",Blinding Flash
4,46565,"[0, 1, 1]",Stance Breaker
6,59636,"[1, 1, 0]",Open Mind
8,30456,"[0, 0, 2]",High Alert
...,...,...,...
994,7809,"[9, 7, 9]",Wild Storm
995,62439,"[9, 9, 7]",Enraged Reaver
996,44566,"[8, 9, 9]",Lightning Rod
998,62034,"[9, 9, 8]",Prism Guard


Get Emotion tiers into the lists now

In [134]:
def get_tiers(crafting_items, blight_items):
    """
    Aggregates the 'Tier' from blight_items for each item ID in crafting_items.
    It assumes blight_items is indexed by item IDs.
    """
    return [blight_items.loc[item_id, 'Tier'] for item_id in crafting_items]

# Apply the function to each row in 'joined_results_crafting'
joined_results_crafting['BlightCraftingItems'] = joined_results_crafting.apply(
    lambda row: get_tiers(row['BlightCraftingItems'], blight_items), axis=1
)
joined_results_crafting

,PassiveSkillGraphId,BlightCraftingItems,Name
0,9736,"[1, 1, 1]",Insulated Treads
2,42354,"[1, 2, 1]",Blinding Flash
4,46565,"[1, 2, 2]",Stance Breaker
6,59636,"[2, 2, 1]",Open Mind
8,30456,"[1, 1, 3]",High Alert
...,...,...,...
994,7809,"[10, 8, 10]",Wild Storm
995,62439,"[10, 10, 8]",Enraged Reaver
996,44566,"[9, 10, 10]",Lightning Rod
998,62034,"[10, 10, 9]",Prism Guard


## Load json data now

This is correct now, but we need to load in the items from the api side of it now

In [135]:
raw_json_data = {
    group['id']: group
    for group 
    in json.loads(
            open(
                f"{cwd}/../json-api/{lang}/stats.json", encoding="utf-8"
            ).read()
        )['result']
    }

get the enchants json data

In [136]:
enchants = raw_json_data["enchant"]
enchants_entries = enchants["entries"]
enchants_entries_filtered = [entry for entry in enchants_entries if entry["id"].startswith("enchant.stat_2954116742")]
enchants_df = pd.DataFrame(enchants_entries_filtered)
enchants_df

,id,text,type
0,enchant.stat_2954116742|30456,Allocates High Alert,enchant
1,enchant.stat_2954116742|57190,Allocates Doomsayer,enchant
2,enchant.stat_2954116742|34300,Allocates Conservative Casting,enchant
3,enchant.stat_2954116742|57388,Allocates Overwhelming Strike,enchant
4,enchant.stat_2954116742|50609,Allocates Hard to Kill,enchant
...,...,...,...
598,enchant.stat_2954116742|44527,Allocates Cautious Concoctions,enchant
599,enchant.stat_2954116742|47635,Allocates Overload,enchant
600,enchant.stat_2954116742|30392,Allocates Succour,enchant
601,enchant.stat_2954116742|4547,Allocates Unnatural Resilience,enchant


In [137]:
enchants_df["value"] = enchants_df["id"].apply(lambda x: x.split("|")[-1]).astype(int)
enchants_df


,id,text,type,value
0,enchant.stat_2954116742|30456,Allocates High Alert,enchant,30456
1,enchant.stat_2954116742|57190,Allocates Doomsayer,enchant,57190
2,enchant.stat_2954116742|34300,Allocates Conservative Casting,enchant,34300
3,enchant.stat_2954116742|57388,Allocates Overwhelming Strike,enchant,57388
4,enchant.stat_2954116742|50609,Allocates Hard to Kill,enchant,50609
...,...,...,...,...
598,enchant.stat_2954116742|44527,Allocates Cautious Concoctions,enchant,44527
599,enchant.stat_2954116742|47635,Allocates Overload,enchant,47635
600,enchant.stat_2954116742|30392,Allocates Succour,enchant,30392
601,enchant.stat_2954116742|4547,Allocates Unnatural Resilience,enchant,4547


## Join enchants_df with results_crafting

In [138]:
output_df = pd.merge(
    joined_results_crafting,
    enchants_df,
    left_on='PassiveSkillGraphId',
    right_on='value',
    suffixes=('_results', '_json')
)[["BlightCraftingItems","Name","text","value"]].sort_values(by="value")
output_df

,BlightCraftingItems,Name,text,value
462,"[4, 3, 10]",Fast Acting Toxins,Allocates Fast Acting Toxins,55
260,"[2, 6, 8]",Insightfulness,Allocates Insightfulness,116
430,"[5, 9, 9]",Storm Swell,Allocates Storm Swell,336
124,"[6, 6, 3]",Heatproofing,Allocates Heatproofing,372
335,"[3, 9, 4]",Natural Immunity,Allocates Natural Immunity,934
...,...,...,...,...
153,"[7, 4, 2]",Titanic,Allocates Titanic,65160
101,"[6, 3, 4]",Viciousness,Allocates Viciousness,65193
478,"[10, 5, 3]",Overflowing Power,Allocates Overflowing Power,65204
50,"[2, 5, 3]",Smooth Buckler,Allocates Smooth Buckler,65265


# form into output json

In [140]:
starting_json = {
    "ref": "Allocates #",
    "better": 0,
    "trade": {
      "ids": {
        "enchant": [
          "enchant.stat_2954116742"
        ]
      },
      "option": True
    }
}
starting_json

{'ref': 'Allocates #',
 'better': 0,
 'trade': {'ids': {'enchant': ['enchant.stat_2954116742']}, 'option': True}}

In [150]:
def row_to_json(row):
    return {
        "string": row["text"],
        "value": row["value"],
        "oils": ",".join(map(str, row["BlightCraftingItems"])),
    }

In [153]:
matchers = output_df.apply(lambda row: row_to_json(row), axis=1).to_list()
starting_json["matchers"] = matchers

starting_json

{'ref': 'Allocates #',
 'better': 0,
 'trade': {'ids': {'enchant': ['enchant.stat_2954116742']}, 'option': True},
 'matchers': [{'string': 'Allocates Fast Acting Toxins',
   'value': 55,
   'oils': '4,3,10'},
  {'string': 'Allocates Insightfulness', 'value': 116, 'oils': '2,6,8'},
  {'string': 'Allocates Storm Swell', 'value': 336, 'oils': '5,9,9'},
  {'string': 'Allocates Heatproofing', 'value': 372, 'oils': '6,6,3'},
  {'string': 'Allocates Natural Immunity', 'value': 934, 'oils': '3,9,4'},
  {'string': 'Allocates Shockwaves', 'value': 1087, 'oils': '3,4,1'},
  {'string': 'Allocates Lust for Power', 'value': 1104, 'oils': '10,2,2'},
  {'string': 'Allocates Urgent Call', 'value': 1169, 'oils': '8,10,9'},
  {'string': 'Allocates Unbending', 'value': 1352, 'oils': '8,7,4'},
  {'string': 'Allocates Spiral into Depression',
   'value': 1546,
   'oils': '5,7,9'},
  {'string': 'Allocates Illuminated Crown', 'value': 1823, 'oils': '9,4,9'},
  {'string': 'Allocates Wellspring', 'value': 2021,